In [1]:
import os 
import sys
import inspect

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, parentdir) 

import pyLLE
import numpy as np
from scipy import constants as cts
import pandas as pd
import pickle as pkl
import plotly.graph_objs as go
import plotly.graph_objs
import plotly.io as pio
import time
pio.renderers.default = "notebook"
pio.templates.default = "seaborn"

************************************************************
Julia solver:
/Users/yakub/Documents/GitHub/pyLLE/pyLLE/ComputeLLE.jl
************************************************************


Comparing the obtained Conversion efficiency from our implementation of bidirectional LLE with the results in Black, Papp: https://opg.optica.org/optica/fulltext.cfm?uri=optica-9-10-1183&id=510242

This will tell us whether the simulations accurately capture the dynamics of a real PhCr system or not

In [2]:
#loop reflector parameters
r=0.0
phi=np.pi
R=np.sqrt(r)*np.exp(1j*phi)
res = dict(
        R=98e-6, 
        Qi=10e6, 
        Qc=10e6, 
        γ=0.46834246334531376, 
        dispfile="./Tantala_test_PhCr_Disp_300MHz.csv", #"./RW1000_H430.csv",
        Bragg=0.3e9 ,#In terms of frequency shift that the pump mode observes due to bragg
        loop_ref_real=np.real(R),
        loop_ref_imag=np.imag(R) #Reflection coefficient induced effectively by the isolator+loop

)

sim = dict(
    Pin=[1e-3], 
    f_pmp=[193.4144890323e12], #pump frequency for wavelength 1550nm
    φ_pmp=[0], 
    δω=[None], 
    Tscan=0.7e6,
    μ_sim=[-50, 50],
    δω_init= 1e9 * 2 * np.pi,
    δω_end= -6.5e9 * 2 * np.pi,
    num_probe = 5000, 
)


In [3]:
solverp1 = pyLLE.LLEsolver(sim=sim, res=res,debug=True)

In [4]:
fig = solverp1.Analyze(plot=True)
fig.update_yaxes(range = [-50, 50], title = 'D<sub>int</sub> (GHz)')
fig.show()

In [5]:
solverp1.disp

+----------------------------------------------------------------------------------------------------------------------+
|  Parameter      Description                         Values                                             Units         |
+----------------------------------------------------------------------------------------------------------------------+
|  ng             [1.899 ... 2.168]                   Group Index                                                      |
|  D              [-1471.386 ... -3557.268]           Dispersion                                         (ps/nm/km)    |
|  neff           [-0.286 ... 0.223]                  Effctive Index                                                   |
|  freq           [168.823 ... 216.442] (THz)         Mode frequency                                     Hz            |
|  freq_sim       [181.630 ... 205.199] (THz)         Mode frequency to match the sim domain             Hz            |
|  Dint           [array for eac

In [6]:
solverp1.sim

+---------------------------------------------------------------------------------------------------------+
|  Parameter          Description         Values                                      Units               |
+---------------------------------------------------------------------------------------------------------+
|  Pin                [1.000] (mW)        Pump power                                  W                   |
|  Tscan              700000.0            Simulation time length                      unit of round trip  |
|  f_pmp              [193.414] (THz)     Pump frequencies                            Hz                  |
|  φ_pmp              [0]                 Pump phase                                  rad                 |
|  f_center           193.414 (THz)       Center of the sim domain                    Hz                  |
|  δω_init            1.000 (GHz)         Start of the frequency sweep                x2π Hz              |
|  δω_end             -6.500

In [7]:
solverp1.res

+----------------------------------------------------------------------------------------------------------+
|  Parameter          Description                             Values                            Units      |
+----------------------------------------------------------------------------------------------------------+
|  R                  98.0                                    Ring radius                       µm         |
|  Qi                 1.000 x10^7                             Intrinsic quality factor                     |
|  Qc                 1.000 x10^7                             Coupling quality factor                      |
|  γ                  0.46834246334531376                     Non-linear coefficient            W^-1 m^-1  |
|  dispfile           ./Tantala_test_PhCr_Disp_300MHz.csv     mode/resonance frequency file                |
|  Bragg              300.0                                   Bragg Shift (MHz)                            |
|  Loop Reflector  

In [8]:
print(f"R = {solverp1.res.R}")
print(f"Pin = {solverp1.sim.Pin}")
print(f"D1 = {solverp1.disp.D1*1e-9/(2*np.pi)} x 2π GHz")

R = 9.8e-05
Pin = [0.001]
D1 = 240.50000000000006 x 2π GHz


In [9]:
solverp1.Setup(verbose = True)
solverp1.SolveTemporal(bin = "/usr/local/bin/julia",verbose=False)
time.sleep(1)
solverp1.RetrieveData()
solverp1.sol

-- Solving standard LLE --
	Simulation Parameters
		R = 98.00 µm
		Qi = 10.00 M
		Qc = 10.00 M
		γ = 0.47 
		Bragg Shift = 300.00 MHz
		loop_ref_real = -0.00 
		loop_ref_imag = 0.00 
	Simulation Parameters
		Pin[0] = 1.00 mW
		f_pmp[0] = 193.41 THz
		Tscan = 0.70 x1e6 Round Trip
		μ_sim = [-50.00,50.00] 
		δω_init = 1.00 x2π GHz
		δω_end = -6.50 x2π GHz
		μ_fit = [None,None] 
		δω_stop = -6.50 x2π GHz
		ind_pump_sweep[0] = 0.00 

HDF5 parameter file can be foud in: /var/folders/k2/n79h9ytx5rq114j_k4jrxv5w0000gn/T/tmpdy1z_ggpParamLLEJulia.h5
----------------------------------------------------------------------
2024-01-25 01:10:39
Computing LLE [**************************************************] 100%

Simulation Time 00h:01min:18.1s
----------------------------------------------------------------------
wtf


+-------------------------------------------------------------------------------------------------+
|  Parameter     Description                          Values                               Units  |
+-------------------------------------------------------------------------------------------------+
|  δfreq         [1.000 ... -6.499] (GHz)             Pump detuning                        Hz     |
|  time          [-0.388 ... 2.522] (μs)              Simualtion time                      s      |
|  Awg           [101 fast time x 5000 slow time]     E. field in time domain in wg,       V/m    |
|  Acav          [101 fast time x 5000 slow time]     E. field in time domain in cav.      V/m    |
|  Ewg           [101 spectra x 5000 slow time]       E. field in freq. domain in wg.      V/m    |
|  Ecav          [101 spectra x 5000 slow time]       E. field in freq. domain in cav.     V/m    |
|  Pcomb         [0.000 ... 0.000] (mW)               Intra-cavity comb power              W      |


In [10]:
fig = go.Figure()

tr = go.Scatter(y=solverp1.sol.Pcomb * 1e3)

fig.add_trace(tr)
fig.update_layout(
    xaxis_title="LLE sub-sampled step", yaxis_title="Intra-cavity Power (mW)"
)
#fig.add_annotation(x=4500, y=4.9, ax=0, ay=-50, text="Single Soliton Step")
# fig.add_annotation(x=3151, y=7.6, ax=0, ay=-50, text="Two Solitons Step")
#fig.add_annotation(x=1931, y=16, ax=0, ay=80, text="Modulation Instability")
#fig.add_annotation(x=500, y=6, ax=-80, ay=-80, text="Primary Comb")
#fig.show()

In [11]:
ind=738
solverp1.PlotCombSpectra(ind).show()

In [12]:
#loop reflector parameters
r=0.0
phi=np.pi
R=np.sqrt(r)*np.exp(1j*phi)
res = dict(
        R=98e-6, 
        Qi=10e6, 
        Qc=10e6, 
        γ=0.46834246334531376, 
        dispfile="./Tantala_test_PhCr_Disp_300MHz.csv", #"./RW1000_H430.csv",
        Bragg=0.3e9 ,#In terms of frequency shift that the pump mode observes due to bragg
        loop_ref_real=np.real(R),
        loop_ref_imag=np.imag(R) #Reflection coefficient induced effectively by the isolator+loop

)

sim = dict(
    Pin=[4e-3], 
    f_pmp=[193.4144890323e12], #pump frequency for wavelength 1550nm
    φ_pmp=[0], 
    δω=[None], 
    Tscan=0.7e6,
    μ_sim=[-50, 50],
    δω_init= 1e9 * 2 * np.pi,
    δω_end= -6.5e9 * 2 * np.pi,
    num_probe = 5000, 
)


In [13]:
solverp2 = pyLLE.LLEsolver(sim=sim, res=res,debug=True)

In [14]:
fig = solverp2.Analyze(plot=True)
fig.update_yaxes(range = [-50, 50], title = 'D<sub>int</sub> (GHz)')
fig.show()

In [15]:
print(f"R = {solverp2.res.R}")
print(f"Pin = {solverp2.sim.Pin}")
print(f"D1 = {solverp2.disp.D1*1e-9/(2*np.pi)} x 2π GHz")

R = 9.8e-05
Pin = [0.004]
D1 = 240.50000000000006 x 2π GHz


In [16]:
solverp2.Setup(verbose = True)
solverp2.SolveTemporal(bin = "/usr/local/bin/julia",verbose=False)
time.sleep(1)
solverp2.RetrieveData()
solverp2.sol

-- Solving standard LLE --
	Simulation Parameters
		R = 98.00 µm
		Qi = 10.00 M
		Qc = 10.00 M
		γ = 0.47 
		Bragg Shift = 300.00 MHz
		loop_ref_real = -0.00 
		loop_ref_imag = 0.00 
	Simulation Parameters
		Pin[0] = 4.00 mW
		f_pmp[0] = 193.41 THz
		Tscan = 0.70 x1e6 Round Trip
		μ_sim = [-50.00,50.00] 
		δω_init = 1.00 x2π GHz
		δω_end = -6.50 x2π GHz
		μ_fit = [None,None] 
		δω_stop = -6.50 x2π GHz
		ind_pump_sweep[0] = 0.00 

HDF5 parameter file can be foud in: /var/folders/k2/n79h9ytx5rq114j_k4jrxv5w0000gn/T/tmpdy1z_ggpParamLLEJulia.h5
----------------------------------------------------------------------
2024-01-25 01:12:04
Computing LLE [**************************************************] 100%

Simulation Time 00h:01min:24.9s
----------------------------------------------------------------------
wtf


+-------------------------------------------------------------------------------------------------+
|  Parameter     Description                          Values                               Units  |
+-------------------------------------------------------------------------------------------------+
|  δfreq         [1.000 ... -6.499] (GHz)             Pump detuning                        Hz     |
|  time          [-0.388 ... 2.522] (μs)              Simualtion time                      s      |
|  Awg           [101 fast time x 5000 slow time]     E. field in time domain in wg,       V/m    |
|  Acav          [101 fast time x 5000 slow time]     E. field in time domain in cav.      V/m    |
|  Ewg           [101 spectra x 5000 slow time]       E. field in freq. domain in wg.      V/m    |
|  Ecav          [101 spectra x 5000 slow time]       E. field in freq. domain in cav.     V/m    |
|  Pcomb         [0.000 ... 0.000] (mW)               Intra-cavity comb power              W      |


In [17]:
fig = go.Figure()

tr = go.Scatter(y=solverp2.sol.Pcomb * 1e3)

fig.add_trace(tr)
fig.update_layout(
    xaxis_title="LLE sub-sampled step", yaxis_title="Intra-cavity Power (mW)"
)
#fig.add_annotation(x=4500, y=4.9, ax=0, ay=-50, text="Single Soliton Step")
# fig.add_annotation(x=3151, y=7.6, ax=0, ay=-50, text="Two Solitons Step")
#fig.add_annotation(x=1931, y=16, ax=0, ay=80, text="Modulation Instability")
#fig.add_annotation(x=500, y=6, ax=-80, ay=-80, text="Primary Comb")
#fig.show()

In [40]:
ind=740
solverp2.PlotCombSpectra(ind).show()

In [19]:
#loop reflector parameters
r=0.0
phi=np.pi
R=np.sqrt(r)*np.exp(1j*phi)
res = dict(
        R=98e-6, 
        Qi=10e6, 
        Qc=10e6, 
        γ=0.46834246334531376, 
        dispfile="./Tantala_test_PhCr_Disp_300MHz.csv", #"./RW1000_H430.csv",
        Bragg=0.3e9 ,#In terms of frequency shift that the pump mode observes due to bragg
        loop_ref_real=np.real(R),
        loop_ref_imag=np.imag(R) #Reflection coefficient induced effectively by the isolator+loop

)

sim = dict(
    Pin=[10e-3], 
    f_pmp=[193.4144890323e12], #pump frequency for wavelength 1550nm
    φ_pmp=[0], 
    δω=[None], 
    Tscan=0.7e6,
    μ_sim=[-50, 50],
    δω_init= 1e9 * 2 * np.pi,
    δω_end= -6.5e9 * 2 * np.pi,
    num_probe = 5000, 
)


In [20]:
solverp3 = pyLLE.LLEsolver(sim=sim, res=res,debug=True)

In [21]:
fig = solverp3.Analyze(plot=True)
fig.update_yaxes(range = [-50, 50], title = 'D<sub>int</sub> (GHz)')
fig.show()

In [22]:
print(f"R = {solverp3.res.R}")
print(f"Pin = {solverp3.sim.Pin}")
print(f"D1 = {solverp3.disp.D1*1e-9/(2*np.pi)} x 2π GHz")

R = 9.8e-05
Pin = [0.01]
D1 = 240.50000000000006 x 2π GHz


In [23]:
solverp3.Setup(verbose = True)
solverp3.SolveTemporal(bin = "/usr/local/bin/julia",verbose=False)
time.sleep(1)
solverp3.RetrieveData()
solverp3.sol

-- Solving standard LLE --
	Simulation Parameters
		R = 98.00 µm
		Qi = 10.00 M
		Qc = 10.00 M
		γ = 0.47 
		Bragg Shift = 300.00 MHz
		loop_ref_real = -0.00 
		loop_ref_imag = 0.00 
	Simulation Parameters
		Pin[0] = 10.00 mW
		f_pmp[0] = 193.41 THz
		Tscan = 0.70 x1e6 Round Trip
		μ_sim = [-50.00,50.00] 
		δω_init = 1.00 x2π GHz
		δω_end = -6.50 x2π GHz
		μ_fit = [None,None] 
		δω_stop = -6.50 x2π GHz
		ind_pump_sweep[0] = 0.00 

HDF5 parameter file can be foud in: /var/folders/k2/n79h9ytx5rq114j_k4jrxv5w0000gn/T/tmpdy1z_ggpParamLLEJulia.h5
----------------------------------------------------------------------
2024-01-25 01:13:36
Computing LLE [************************************************* ] 99%

Simulation Time 00h:01min:17.5s
----------------------------------------------------------------------
wtf


+-------------------------------------------------------------------------------------------------+
|  Parameter     Description                          Values                               Units  |
+-------------------------------------------------------------------------------------------------+
|  δfreq         [1.000 ... -6.499] (GHz)             Pump detuning                        Hz     |
|  time          [-0.388 ... 2.522] (μs)              Simualtion time                      s      |
|  Awg           [101 fast time x 5000 slow time]     E. field in time domain in wg,       V/m    |
|  Acav          [101 fast time x 5000 slow time]     E. field in time domain in cav.      V/m    |
|  Ewg           [101 spectra x 5000 slow time]       E. field in freq. domain in wg.      V/m    |
|  Ecav          [101 spectra x 5000 slow time]       E. field in freq. domain in cav.     V/m    |
|  Pcomb         [0.000 ... 0.000] (mW)               Intra-cavity comb power              W      |


In [24]:
fig = go.Figure()

tr = go.Scatter(y=solverp3.sol.Pcomb * 1e3)

fig.add_trace(tr)
fig.update_layout(
    xaxis_title="LLE sub-sampled step", yaxis_title="Intra-cavity Power (mW)"
)
#fig.add_annotation(x=4500, y=4.9, ax=0, ay=-50, text="Single Soliton Step")
# fig.add_annotation(x=3151, y=7.6, ax=0, ay=-50, text="Two Solitons Step")
#fig.add_annotation(x=1931, y=16, ax=0, ay=80, text="Modulation Instability")
#fig.add_annotation(x=500, y=6, ax=-80, ay=-80, text="Primary Comb")
#fig.show()

In [25]:
ind=750
solverp3.PlotCombSpectra(ind).show()

In [26]:
#loop reflector parameters
r=0.0
phi=np.pi
R=np.sqrt(r)*np.exp(1j*phi)
res = dict(
        R=98e-6, 
        Qi=10e6, 
        Qc=10e6, 
        γ=0.46834246334531376, 
        dispfile="./Tantala_test_PhCr_Disp_300MHz.csv", #"./RW1000_H430.csv",
        Bragg=0.3e9 ,#In terms of frequency shift that the pump mode observes due to bragg
        loop_ref_real=np.real(R),
        loop_ref_imag=np.imag(R) #Reflection coefficient induced effectively by the isolator+loop

)

sim = dict(
    Pin=[20e-3], 
    f_pmp=[193.4144890323e12], #pump frequency for wavelength 1550nm
    φ_pmp=[0], 
    δω=[None], 
    Tscan=0.7e6,
    μ_sim=[-50, 50],
    δω_init= 1e9 * 2 * np.pi,
    δω_end= -6.5e9 * 2 * np.pi,
    num_probe = 5000, 
)

solverp4 = pyLLE.LLEsolver(sim=sim, res=res,debug=True)
fig = solverp4.Analyze(plot=False)
print(f"R = {solverp4.res.R}")
print(f"Pin = {solverp4.sim.Pin}")
print(f"D1 = {solverp4.disp.D1*1e-9/(2*np.pi)} x 2π GHz")
solverp4.Setup(verbose = True)
solverp4.SolveTemporal(bin = "/usr/local/bin/julia",verbose=False)
time.sleep(1)
solverp4.RetrieveData()
solverp4.sol

fig = go.Figure()

tr = go.Scatter(y=solverp4.sol.Pcomb * 1e3)

fig.add_trace(tr)
fig.update_layout(
    xaxis_title="LLE sub-sampled step", yaxis_title="Intra-cavity Power (mW)"
)
#fig.add_annotation(x=4500, y=4.9, ax=0, ay=-50, text="Single Soliton Step")
# fig.add_annotation(x=3151, y=7.6, ax=0, ay=-50, text="Two Solitons Step")
#fig.add_annotation(x=1931, y=16, ax=0, ay=80, text="Modulation Instability")
#fig.add_annotation(x=500, y=6, ax=-80, ay=-80, text="Primary Comb")
#fig.show()

R = 9.8e-05
Pin = [0.02]
D1 = 240.50000000000006 x 2π GHz
-- Solving standard LLE --
	Simulation Parameters
		R = 98.00 µm
		Qi = 10.00 M
		Qc = 10.00 M
		γ = 0.47 
		Bragg Shift = 300.00 MHz
		loop_ref_real = -0.00 
		loop_ref_imag = 0.00 
	Simulation Parameters
		Pin[0] = 20.00 mW
		f_pmp[0] = 193.41 THz
		Tscan = 0.70 x1e6 Round Trip
		μ_sim = [-50.00,50.00] 
		δω_init = 1.00 x2π GHz
		δω_end = -6.50 x2π GHz
		μ_fit = [None,None] 
		δω_stop = -6.50 x2π GHz
		ind_pump_sweep[0] = 0.00 

HDF5 parameter file can be foud in: /var/folders/k2/n79h9ytx5rq114j_k4jrxv5w0000gn/T/tmpdy1z_ggpParamLLEJulia.h5
----------------------------------------------------------------------
2024-01-25 01:15:01
Computing LLE [**************************************************] 100%

Simulation Time 00h:01min:23.2s
----------------------------------------------------------------------
wtf


In [27]:
ind=965
solverp4.PlotCombSpectra(ind).show()

In [28]:
#loop reflector parameters
r=0.0
phi=np.pi
R=np.sqrt(r)*np.exp(1j*phi)
res = dict(
        R=98e-6, 
        Qi=10e6, 
        Qc=10e6, 
        γ=0.46834246334531376, 
        dispfile="./Tantala_test_PhCr_Disp_300MHz.csv", #"./RW1000_H430.csv",
        Bragg=0.3e9 ,#In terms of frequency shift that the pump mode observes due to bragg
        loop_ref_real=np.real(R),
        loop_ref_imag=np.imag(R) #Reflection coefficient induced effectively by the isolator+loop

)

sim = dict(
    Pin=[20e-3], 
    f_pmp=[193.4144890323e12], #pump frequency for wavelength 1550nm
    φ_pmp=[0], 
    δω=[None], 
    Tscan=0.7e6,
    μ_sim=[-50, 50],
    δω_init= 1e9 * 2 * np.pi,
    δω_end= -6.5e9 * 2 * np.pi,
    num_probe = 5000, 
)

solverp5 = pyLLE.LLEsolver(sim=sim, res=res,debug=True)
fig = solverp5.Analyze(plot=False)
print(f"R = {solverp5.res.R}")
print(f"Pin = {solverp5.sim.Pin}")
print(f"D1 = {solverp5.disp.D1*1e-9/(2*np.pi)} x 2π GHz")
solverp5.Setup(verbose = True)
solverp5.SolveTemporal(bin = "/usr/local/bin/julia",verbose=False)
time.sleep(1)
solverp5.RetrieveData()
solverp5.sol

fig = go.Figure()

tr = go.Scatter(y=solverp5.sol.Pcomb * 1e3)

fig.add_trace(tr)
fig.update_layout(
    xaxis_title="LLE sub-sampled step", yaxis_title="Intra-cavity Power (mW)"
)
#fig.add_annotation(x=4500, y=4.9, ax=0, ay=-50, text="Single Soliton Step")
# fig.add_annotation(x=3151, y=7.6, ax=0, ay=-50, text="Two Solitons Step")
#fig.add_annotation(x=1931, y=16, ax=0, ay=80, text="Modulation Instability")
#fig.add_annotation(x=500, y=6, ax=-80, ay=-80, text="Primary Comb")
#fig.show()

R = 9.8e-05
Pin = [0.02]
D1 = 240.50000000000006 x 2π GHz
-- Solving standard LLE --
	Simulation Parameters
		R = 98.00 µm
		Qi = 10.00 M
		Qc = 10.00 M
		γ = 0.47 
		Bragg Shift = 300.00 MHz
		loop_ref_real = -0.00 
		loop_ref_imag = 0.00 
	Simulation Parameters
		Pin[0] = 20.00 mW
		f_pmp[0] = 193.41 THz
		Tscan = 0.70 x1e6 Round Trip
		μ_sim = [-50.00,50.00] 
		δω_init = 1.00 x2π GHz
		δω_end = -6.50 x2π GHz
		μ_fit = [None,None] 
		δω_stop = -6.50 x2π GHz
		ind_pump_sweep[0] = 0.00 

HDF5 parameter file can be foud in: /var/folders/k2/n79h9ytx5rq114j_k4jrxv5w0000gn/T/tmpdy1z_ggpParamLLEJulia.h5
----------------------------------------------------------------------
2024-01-25 01:16:32
Computing LLE [**************************************************] 100%

Simulation Time 00h:01min:21.3s
----------------------------------------------------------------------
wtf


In [29]:
ind=965
solverp5.PlotCombSpectra(ind).show()

In [30]:
#loop reflector parameters
r=0.0
phi=np.pi
R=np.sqrt(r)*np.exp(1j*phi)
res = dict(
        R=98e-6, 
        Qi=10e6, 
        Qc=10e6, 
        γ=0.46834246334531376, 
        dispfile="./Tantala_test_PhCr_Disp_300MHz.csv", #"./RW1000_H430.csv",
        Bragg=0.3e9 ,#In terms of frequency shift that the pump mode observes due to bragg
        loop_ref_real=np.real(R),
        loop_ref_imag=np.imag(R) #Reflection coefficient induced effectively by the isolator+loop

)

sim = dict(
    Pin=[50e-3], 
    f_pmp=[193.4144890323e12], #pump frequency for wavelength 1550nm
    φ_pmp=[0], 
    δω=[None], 
    Tscan=0.7e6,
    μ_sim=[-50, 50],
    δω_init= 1e9 * 2 * np.pi,
    δω_end= -6.5e9 * 2 * np.pi,
    num_probe = 5000, 
)

solverp6 = pyLLE.LLEsolver(sim=sim, res=res,debug=True)
fig = solverp6.Analyze(plot=False)
print(f"R = {solverp6.res.R}")
print(f"Pin = {solverp6.sim.Pin}")
print(f"D1 = {solverp6.disp.D1*1e-9/(2*np.pi)} x 2π GHz")
solverp6.Setup(verbose = True)
solverp6.SolveTemporal(bin = "/usr/local/bin/julia",verbose=False)
time.sleep(1)
solverp6.RetrieveData()
solverp6.sol

fig = go.Figure()

tr = go.Scatter(y=solverp6.sol.Pcomb * 1e3)

fig.add_trace(tr)
fig.update_layout(
    xaxis_title="LLE sub-sampled step", yaxis_title="Intra-cavity Power (mW)"
)
#fig.add_annotation(x=4500, y=4.9, ax=0, ay=-50, text="Single Soliton Step")
# fig.add_annotation(x=3151, y=7.6, ax=0, ay=-50, text="Two Solitons Step")
#fig.add_annotation(x=1931, y=16, ax=0, ay=80, text="Modulation Instability")
#fig.add_annotation(x=500, y=6, ax=-80, ay=-80, text="Primary Comb")
#fig.show()

R = 9.8e-05
Pin = [0.05]
D1 = 240.50000000000006 x 2π GHz
-- Solving standard LLE --
	Simulation Parameters
		R = 98.00 µm
		Qi = 10.00 M
		Qc = 10.00 M
		γ = 0.47 
		Bragg Shift = 300.00 MHz
		loop_ref_real = -0.00 
		loop_ref_imag = 0.00 
	Simulation Parameters
		Pin[0] = 50.00 mW
		f_pmp[0] = 193.41 THz
		Tscan = 0.70 x1e6 Round Trip
		μ_sim = [-50.00,50.00] 
		δω_init = 1.00 x2π GHz
		δω_end = -6.50 x2π GHz
		μ_fit = [None,None] 
		δω_stop = -6.50 x2π GHz
		ind_pump_sweep[0] = 0.00 

HDF5 parameter file can be foud in: /var/folders/k2/n79h9ytx5rq114j_k4jrxv5w0000gn/T/tmpdy1z_ggpParamLLEJulia.h5
----------------------------------------------------------------------
2024-01-25 01:18:01
Computing LLE [************************************************* ] 99%

Simulation Time 00h:01min:45.7s
----------------------------------------------------------------------
wtf


In [31]:
ind=965
solverp6.PlotCombSpectra(ind).show()

In [32]:
#loop reflector parameters
r=0.0
phi=np.pi
R=np.sqrt(r)*np.exp(1j*phi)
res = dict(
        R=98e-6, 
        Qi=10e6, 
        Qc=10e6, 
        γ=0.46834246334531376, 
        dispfile="./Tantala_test_PhCr_Disp_300MHz.csv", #"./RW1000_H430.csv",
        Bragg=0.3e9 ,#In terms of frequency shift that the pump mode observes due to bragg
        loop_ref_real=np.real(R),
        loop_ref_imag=np.imag(R) #Reflection coefficient induced effectively by the isolator+loop

)

sim = dict(
    Pin=[100e-3], 
    f_pmp=[193.4144890323e12], #pump frequency for wavelength 1550nm
    φ_pmp=[0], 
    δω=[None], 
    Tscan=0.7e6,
    μ_sim=[-50, 50],
    δω_init= 1e9 * 2 * np.pi,
    δω_end= -6.5e9 * 2 * np.pi,
    num_probe = 5000, 
)

solverp7 = pyLLE.LLEsolver(sim=sim, res=res,debug=True)
fig = solverp7.Analyze(plot=False)
print(f"R = {solverp7.res.R}")
print(f"Pin = {solverp7.sim.Pin}")
print(f"D1 = {solverp7.disp.D1*1e-9/(2*np.pi)} x 2π GHz")
solverp7.Setup(verbose = True)
solverp7.SolveTemporal(bin = "/usr/local/bin/julia",verbose=False)
time.sleep(1)
solverp7.RetrieveData()
solverp7.sol

fig = go.Figure()

tr = go.Scatter(y=solverp7.sol.Pcomb * 1e3)

fig.add_trace(tr)
fig.update_layout(
    xaxis_title="LLE sub-sampled step", yaxis_title="Intra-cavity Power (mW)"
)
#fig.add_annotation(x=4500, y=4.9, ax=0, ay=-50, text="Single Soliton Step")
# fig.add_annotation(x=3151, y=7.6, ax=0, ay=-50, text="Two Solitons Step")
#fig.add_annotation(x=1931, y=16, ax=0, ay=80, text="Modulation Instability")
#fig.add_annotation(x=500, y=6, ax=-80, ay=-80, text="Primary Comb")
#fig.show()

R = 9.8e-05
Pin = [0.1]
D1 = 240.50000000000006 x 2π GHz
-- Solving standard LLE --
	Simulation Parameters
		R = 98.00 µm
		Qi = 10.00 M
		Qc = 10.00 M
		γ = 0.47 
		Bragg Shift = 300.00 MHz
		loop_ref_real = -0.00 
		loop_ref_imag = 0.00 
	Simulation Parameters
		Pin[0] = 100.00 mW
		f_pmp[0] = 193.41 THz
		Tscan = 0.70 x1e6 Round Trip
		μ_sim = [-50.00,50.00] 
		δω_init = 1.00 x2π GHz
		δω_end = -6.50 x2π GHz
		μ_fit = [None,None] 
		δω_stop = -6.50 x2π GHz
		ind_pump_sweep[0] = 0.00 

HDF5 parameter file can be foud in: /var/folders/k2/n79h9ytx5rq114j_k4jrxv5w0000gn/T/tmpdy1z_ggpParamLLEJulia.h5
----------------------------------------------------------------------
2024-01-25 01:19:54
Computing LLE [**************************************************] 100%

Simulation Time 00h:02min:00.8s
----------------------------------------------------------------------
wtf


In [33]:
ind=965
solverp7.PlotCombSpectra(ind).show()

In [34]:
#loop reflector parameters
r=0.0
phi=np.pi
R=np.sqrt(r)*np.exp(1j*phi)
res = dict(
        R=98e-6, 
        Qi=10e6, 
        Qc=10e6, 
        γ=0.46834246334531376, 
        dispfile="./Tantala_test_PhCr_Disp_300MHz.csv", #"./RW1000_H430.csv",
        Bragg=0.3e9 ,#In terms of frequency shift that the pump mode observes due to bragg
        loop_ref_real=np.real(R),
        loop_ref_imag=np.imag(R) #Reflection coefficient induced effectively by the isolator+loop

)

sim = dict(
    Pin=[200e-3], 
    f_pmp=[193.4144890323e12], #pump frequency for wavelength 1550nm
    φ_pmp=[0], 
    δω=[None], 
    Tscan=0.7e6,
    μ_sim=[-50, 50],
    δω_init= 1e9 * 2 * np.pi,
    δω_end= -6.5e9 * 2 * np.pi,
    num_probe = 5000, 
)

solverp8 = pyLLE.LLEsolver(sim=sim, res=res,debug=True)
fig = solverp8.Analyze(plot=False)
print(f"R = {solverp8.res.R}")
print(f"Pin = {solverp8.sim.Pin}")
print(f"D1 = {solverp8.disp.D1*1e-9/(2*np.pi)} x 2π GHz")
solverp8.Setup(verbose = True)
solverp8.SolveTemporal(bin = "/usr/local/bin/julia",verbose=False)
time.sleep(1)
solverp8.RetrieveData()
solverp8.sol

fig = go.Figure()

tr = go.Scatter(y=solverp8.sol.Pcomb * 1e3)

fig.add_trace(tr)
fig.update_layout(
    xaxis_title="LLE sub-sampled step", yaxis_title="Intra-cavity Power (mW)"
)
#fig.add_annotation(x=4500, y=4.9, ax=0, ay=-50, text="Single Soliton Step")
# fig.add_annotation(x=3151, y=7.6, ax=0, ay=-50, text="Two Solitons Step")
#fig.add_annotation(x=1931, y=16, ax=0, ay=80, text="Modulation Instability")
#fig.add_annotation(x=500, y=6, ax=-80, ay=-80, text="Primary Comb")
#fig.show()

R = 9.8e-05
Pin = [0.2]
D1 = 240.50000000000006 x 2π GHz
-- Solving standard LLE --
	Simulation Parameters
		R = 98.00 µm
		Qi = 10.00 M
		Qc = 10.00 M
		γ = 0.47 
		Bragg Shift = 300.00 MHz
		loop_ref_real = -0.00 
		loop_ref_imag = 0.00 
	Simulation Parameters
		Pin[0] = 200.00 mW
		f_pmp[0] = 193.41 THz
		Tscan = 0.70 x1e6 Round Trip
		μ_sim = [-50.00,50.00] 
		δω_init = 1.00 x2π GHz
		δω_end = -6.50 x2π GHz
		μ_fit = [None,None] 
		δω_stop = -6.50 x2π GHz
		ind_pump_sweep[0] = 0.00 

HDF5 parameter file can be foud in: /var/folders/k2/n79h9ytx5rq114j_k4jrxv5w0000gn/T/tmpdy1z_ggpParamLLEJulia.h5
----------------------------------------------------------------------
2024-01-25 01:22:02
Computing LLE [**************************************************] 100%

Simulation Time 00h:01min:43.2s
----------------------------------------------------------------------
wtf


In [37]:
ind=1700
solverp8.PlotCombSpectra(ind).show()